In [154]:
import os
import pandas as pd 
import torch.nn as nn
import torch
import torch.nn.functional as F

# Data Pre-Processing

### Transfroming the data into more readable input to the model

In [155]:
def encode_song(song):
    result = []
    prev = {'note0': -1, 'note1': -1, 'note2': -1, 'note3': -1}
    for index, row in song.iterrows():
        frame = []
        for voice in ['note0', 'note1', 'note2', 'note3']:
            pitch = row[voice]
            previous_pitch = prev[voice]
            
            tied = 1 if pitch == previous_pitch else 0
            frame.append((int(pitch), tied))
            prev[voice] = pitch
        result.append(frame)
    tensor = torch.tensor(result, dtype=torch.int64)
    return tensor

In [205]:
folder_path = 'Data/'
test = []
train = []
validation = []
for dirname in os.listdir(folder_path):
    if dirname != '.DS_Store':
        for filename in os.listdir(folder_path + dirname):
            if filename != '.ipynb_checkpoints':
                df = pd.read_csv(folder_path + dirname + '/' + filename)
                song = encode_song(df)
                if dirname == 'test':
                    test.append(song)
                if dirname == 'train':
                    train.append(song)
                if dirname == 'valid':
                    validation.append(song)

In [224]:
class Model(nn.Module):
    def __init__(self, input_size=2, hidden_size=128, output_size=127, num_harmony_parts=3, num_layers=1):
        super(Model, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size * num_harmony_parts)  # Predict pitch classes for harmony parts
        self.num_harmony_parts = num_harmony_parts
        self.output_size = output_size

    def forward(self, x):        
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out)
        
        output = output.view(x.size(0), -1, self.num_harmony_parts, self.output_size)
        return output

In [254]:
def train_model(model, optimizer, criterion, num_epochs):
    model.train()

    for song_index, song in enumerate(train):
        print(f"Training on song {song_index + 1}")
        
        melody = song[:, 0, :].unsqueeze(0).float()
        harmonies = song[:, 1:, :].permute(1, 0, 2).float()
        harmonies = harmonies[:, :, 0] 
        harmonies = harmonies.permute(1, 0).unsqueeze(0)
        hidden = None
        for epoch in range(num_epochs):
            optimizer.zero_grad()
            output = model(melody)
            loss = criterion(output.view(-1, 127), harmonies.view(-1).long())
            loss.backward()
            optimizer.step()
            
            if (epoch + 1) % 10 == 0:
                print(f"Song {song_index + 1}, Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")
        
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for val_song in validation:
                val_melody = val_song[:, 0, :].unsqueeze(0).float()
                val_harmonies = val_song[:, 1:, :].permute(1, 0, 2).float() 
                val_harmonies = val_harmonies[:, :, 0]
                val_harmonies = val_harmonies.permute(1, 0).unsqueeze(0)

                val_output = model(val_melody)
                
                val_loss = criterion(val_output.view(-1, 127), val_harmonies.view(-1).long())
                total_val_loss += val_loss.item()

        average_val_loss = total_val_loss / len(validation)
        print(f"Validation Loss after song {song_index + 1}: {average_val_loss}")

        model.train()

In [ ]:
def harmonies_to_class(harmonies):
    harmonies_classes = torch.round(harmonies * 127).long()  
    return harmonies_classes

criterion = torch.nn.CrossEntropyLoss()
num_epochs = 100

model = Model(2, 3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
train_model(model, optimizer, criterion, num_epochs)

Training on song 1
Song 1, Epoch 10/100, Loss: 4.948286056518555
Song 1, Epoch 20/100, Loss: 4.911983966827393
Song 1, Epoch 30/100, Loss: 4.875833988189697
Song 1, Epoch 40/100, Loss: 4.839839458465576
Song 1, Epoch 50/100, Loss: 4.803967475891113
Song 1, Epoch 60/100, Loss: 4.768085479736328
Song 1, Epoch 70/100, Loss: 4.731705188751221
Song 1, Epoch 80/100, Loss: 4.694376468658447
Song 1, Epoch 90/100, Loss: 4.660025119781494
Song 1, Epoch 100/100, Loss: 4.625687599182129
Validation Loss after song 1: 4.728029153285882
Training on song 2
Song 2, Epoch 10/100, Loss: 4.670993804931641
Song 2, Epoch 20/100, Loss: 4.624284267425537
Song 2, Epoch 30/100, Loss: 4.57839822769165
Song 2, Epoch 40/100, Loss: 4.534287452697754
Song 2, Epoch 50/100, Loss: 4.491841793060303
Song 2, Epoch 60/100, Loss: 4.450815677642822
Song 2, Epoch 70/100, Loss: 4.410981178283691
Song 2, Epoch 80/100, Loss: 4.372155666351318
Song 2, Epoch 90/100, Loss: 4.334200382232666
Song 2, Epoch 100/100, Loss: 4.297012329

Song 16, Epoch 40/100, Loss: 2.647670269012451
Song 16, Epoch 50/100, Loss: 2.6263427734375
Song 16, Epoch 60/100, Loss: 2.606395959854126
Song 16, Epoch 70/100, Loss: 2.587783098220825
Song 16, Epoch 80/100, Loss: 2.5704095363616943
Song 16, Epoch 90/100, Loss: 2.5541744232177734
Song 16, Epoch 100/100, Loss: 2.5389840602874756
Validation Loss after song 16: 2.855807738426404
Training on song 17
Song 17, Epoch 10/100, Loss: 2.603363513946533
Song 17, Epoch 20/100, Loss: 2.589007616043091
Song 17, Epoch 30/100, Loss: 2.572453737258911
Song 17, Epoch 40/100, Loss: 2.555948257446289
Song 17, Epoch 50/100, Loss: 2.540196418762207
Song 17, Epoch 60/100, Loss: 2.5253734588623047
Song 17, Epoch 70/100, Loss: 2.5114800930023193
Song 17, Epoch 80/100, Loss: 2.498461961746216
Song 17, Epoch 90/100, Loss: 2.4862546920776367
Song 17, Epoch 100/100, Loss: 2.47479248046875
Validation Loss after song 17: 2.8570422453758044
Training on song 18
Song 18, Epoch 10/100, Loss: 3.0922999382019043
Song 18, 

Song 31, Epoch 60/100, Loss: 2.4431467056274414
Song 31, Epoch 70/100, Loss: 2.4180071353912354
Song 31, Epoch 80/100, Loss: 2.3948426246643066
Song 31, Epoch 90/100, Loss: 2.3734869956970215
Song 31, Epoch 100/100, Loss: 2.353783130645752
Validation Loss after song 31: 2.735662478667039
Training on song 32
Song 32, Epoch 10/100, Loss: 2.5231199264526367
Song 32, Epoch 20/100, Loss: 2.493014097213745
Song 32, Epoch 30/100, Loss: 2.4584639072418213
Song 32, Epoch 40/100, Loss: 2.4247050285339355
Song 32, Epoch 50/100, Loss: 2.393242120742798
Song 32, Epoch 60/100, Loss: 2.3643405437469482
Song 32, Epoch 70/100, Loss: 2.337885856628418
Song 32, Epoch 80/100, Loss: 2.3136627674102783
Song 32, Epoch 90/100, Loss: 2.2914481163024902
Song 32, Epoch 100/100, Loss: 2.2710378170013428
Validation Loss after song 32: 2.7402623188801303
Training on song 33
Song 33, Epoch 10/100, Loss: 2.3785958290100098
Song 33, Epoch 20/100, Loss: 2.3587095737457275
Song 33, Epoch 30/100, Loss: 2.3357696533203125

Song 46, Epoch 70/100, Loss: 2.557823419570923
Song 46, Epoch 80/100, Loss: 2.5421810150146484
Song 46, Epoch 90/100, Loss: 2.527611494064331
Song 46, Epoch 100/100, Loss: 2.5140247344970703
Validation Loss after song 46: 2.732833813398312
Training on song 47
Song 47, Epoch 10/100, Loss: 3.0638034343719482
Song 47, Epoch 20/100, Loss: 3.0044050216674805
Song 47, Epoch 30/100, Loss: 2.941237211227417
Song 47, Epoch 40/100, Loss: 2.880603075027466
Song 47, Epoch 50/100, Loss: 2.824110984802246
Song 47, Epoch 60/100, Loss: 2.771907091140747
Song 47, Epoch 70/100, Loss: 2.7237164974212646
Song 47, Epoch 80/100, Loss: 2.6791751384735107
Song 47, Epoch 90/100, Loss: 2.637930393218994
Song 47, Epoch 100/100, Loss: 2.5996627807617188
Validation Loss after song 47: 2.7351823036487284
Training on song 48
Song 48, Epoch 10/100, Loss: 2.810214042663574
Song 48, Epoch 20/100, Loss: 2.7848877906799316
Song 48, Epoch 30/100, Loss: 2.7574210166931152
Song 48, Epoch 40/100, Loss: 2.7305350303649902
Son

Song 61, Epoch 80/100, Loss: 2.2138397693634033
Song 61, Epoch 90/100, Loss: 2.1941394805908203
Song 61, Epoch 100/100, Loss: 2.175922393798828
Validation Loss after song 61: 2.7810963422824173
Training on song 62
Song 62, Epoch 10/100, Loss: 2.468785047531128
Song 62, Epoch 20/100, Loss: 2.4354379177093506
Song 62, Epoch 30/100, Loss: 2.3987607955932617
Song 62, Epoch 40/100, Loss: 2.3632142543792725
Song 62, Epoch 50/100, Loss: 2.3300442695617676
Song 62, Epoch 60/100, Loss: 2.2994496822357178
Song 62, Epoch 70/100, Loss: 2.2713141441345215
Song 62, Epoch 80/100, Loss: 2.2454400062561035
Song 62, Epoch 90/100, Loss: 2.221621513366699
Song 62, Epoch 100/100, Loss: 2.1996688842773438
Validation Loss after song 62: 2.7325642047784267
Training on song 63
Song 63, Epoch 10/100, Loss: 2.7359187602996826
Song 63, Epoch 20/100, Loss: 2.7040328979492188
Song 63, Epoch 30/100, Loss: 2.667682409286499
Song 63, Epoch 40/100, Loss: 2.6316983699798584
Song 63, Epoch 50/100, Loss: 2.597574710845947

Song 76, Epoch 90/100, Loss: 2.34716534614563
Song 76, Epoch 100/100, Loss: 2.324225425720215
Validation Loss after song 76: 2.713687688876421
Training on song 77
Song 77, Epoch 10/100, Loss: 2.6051313877105713
Song 77, Epoch 20/100, Loss: 2.568634033203125
Song 77, Epoch 30/100, Loss: 2.5257742404937744
Song 77, Epoch 40/100, Loss: 2.4831743240356445
Song 77, Epoch 50/100, Loss: 2.442868709564209
Song 77, Epoch 60/100, Loss: 2.405336856842041
Song 77, Epoch 70/100, Loss: 2.3705532550811768
Song 77, Epoch 80/100, Loss: 2.3383426666259766
Song 77, Epoch 90/100, Loss: 2.3084983825683594
Song 77, Epoch 100/100, Loss: 2.2808196544647217
Validation Loss after song 77: 2.7400708320813303
Training on song 78
Song 78, Epoch 10/100, Loss: 2.60282564163208
Song 78, Epoch 20/100, Loss: 2.587029218673706
Song 78, Epoch 30/100, Loss: 2.566955327987671
Song 78, Epoch 40/100, Loss: 2.551201581954956
Song 78, Epoch 50/100, Loss: 2.5362155437469482
Song 78, Epoch 60/100, Loss: 2.5222644805908203
Song 7

Song 91, Epoch 100/100, Loss: 2.1326587200164795
Validation Loss after song 91: 2.7609691681006017
Training on song 92
Song 92, Epoch 10/100, Loss: 2.866286277770996
Song 92, Epoch 20/100, Loss: 2.7910096645355225
Song 92, Epoch 30/100, Loss: 2.7090227603912354
Song 92, Epoch 40/100, Loss: 2.630509614944458
Song 92, Epoch 50/100, Loss: 2.558026075363159
Song 92, Epoch 60/100, Loss: 2.491838216781616
Song 92, Epoch 70/100, Loss: 2.431580066680908
Song 92, Epoch 80/100, Loss: 2.3767569065093994
Song 92, Epoch 90/100, Loss: 2.3268890380859375
Song 92, Epoch 100/100, Loss: 2.2815394401550293
Validation Loss after song 92: 2.772226095199585
Training on song 93
Song 93, Epoch 10/100, Loss: 2.5142781734466553
Song 93, Epoch 20/100, Loss: 2.487945795059204
Song 93, Epoch 30/100, Loss: 2.4581892490386963
Song 93, Epoch 40/100, Loss: 2.4296534061431885
Song 93, Epoch 50/100, Loss: 2.4036478996276855
Song 93, Epoch 60/100, Loss: 2.3803517818450928
Song 93, Epoch 70/100, Loss: 2.359593391418457
So

Song 106, Epoch 100/100, Loss: 2.5247535705566406
Validation Loss after song 106: 2.6886972280649037
Training on song 107
Song 107, Epoch 10/100, Loss: 2.6957859992980957
Song 107, Epoch 20/100, Loss: 2.6572465896606445
Song 107, Epoch 30/100, Loss: 2.6157612800598145
Song 107, Epoch 40/100, Loss: 2.575944423675537
Song 107, Epoch 50/100, Loss: 2.5392704010009766
Song 107, Epoch 60/100, Loss: 2.5059499740600586
Song 107, Epoch 70/100, Loss: 2.4758033752441406
Song 107, Epoch 80/100, Loss: 2.4485526084899902
Song 107, Epoch 90/100, Loss: 2.423912763595581
Song 107, Epoch 100/100, Loss: 2.401618719100952
Validation Loss after song 107: 2.64756178855896
Training on song 108
Song 108, Epoch 10/100, Loss: 2.575404167175293
Song 108, Epoch 20/100, Loss: 2.5347278118133545
Song 108, Epoch 30/100, Loss: 2.4884703159332275
Song 108, Epoch 40/100, Loss: 2.443620443344116
Song 108, Epoch 50/100, Loss: 2.402135133743286
Song 108, Epoch 60/100, Loss: 2.364323854446411
Song 108, Epoch 70/100, Loss: 

In [245]:
import torch
import torch.nn as nn

# Define the model output size as 127 (for each note class)
# Assuming model output shape: (batch_size, seq_len, 3, 127)
output = torch.randn(5, 10, 3, 127)  # Random output for demonstration
print(output)

# Assume target is structured as note indices for alto, tenor, and bass
# Shape of target: (batch_size, seq_len, 3) - with each entry as an index in [0, 126]
target = torch.randint(0, 127, (5, 10, 3))  # Random target for demonstration
print(target)

# CrossEntropyLoss expects shape (N, C) for input and (N) for target, so reshape
criterion = nn.CrossEntropyLoss()

# Reshape output to (batch_size * seq_len * 3, 127) and target to (batch_size * seq_len * 3)
loss = criterion(output.view(-1, 127), target.view(-1))
print(loss)

tensor([[[[ 6.1264e-01,  1.2438e+00, -6.9005e-01,  ..., -3.5063e-01,
            7.5341e-01, -1.4020e-01],
          [ 5.9260e-01, -2.1966e-01,  4.6525e-01,  ...,  2.9603e-01,
           -1.3076e+00,  4.5744e-01],
          [ 5.0970e-02,  9.2430e-01,  6.1803e-01,  ...,  2.2716e-01,
           -3.9026e-01,  4.9559e-01]],

         [[ 7.6975e-01,  9.2200e-01,  5.6193e-01,  ..., -7.6454e-03,
           -8.0152e-01,  6.9461e-01],
          [-5.4142e-01,  2.2100e+00,  8.0765e-01,  ..., -9.1499e-01,
            4.1876e-01,  4.2862e-01],
          [ 1.8347e+00,  8.9746e-01,  2.4368e+00,  ...,  4.1120e-01,
           -5.6740e-01,  1.0640e-01]],

         [[ 1.1443e+00,  5.1425e-01, -7.5979e-01,  ...,  8.9607e-01,
            5.6485e-01,  1.9133e-01],
          [-7.2332e-01,  1.5633e+00,  7.0833e-01,  ..., -6.8861e-02,
           -4.8263e-01, -1.0481e+00],
          [-8.2456e-01, -8.5643e-01,  1.0783e-01,  ..., -1.2720e-01,
            8.2160e-01, -1.8054e+00]],

         ...,

         [[ 3.32